In [297]:
import pandas as pd
test_data = pd.read_csv('../sample_datasets/x_test.csv')
actual_target = pd.read_csv('../sample_datasets/y_test.csv')

In [298]:
X = test_data[:20]

In [299]:
Y = actual_target[:20]

In [300]:
from art.attacks import ZooAttack
from art.classifiers import XGBoostClassifier
from DataTransform import DataTransform
import numpy as np

In [301]:
transformed_data = DataTransform().transform(X)

In [302]:
import pickle
filename = "XGBoostModel.pkl"
#loading modules
XGBoost_model = pickle.load(open(filename, 'rb'))

In [303]:
results = XGBoost_model.predict(transformed_data)

In [304]:
transformed_data.shape

(20, 19)

In [305]:
classifier = XGBoostClassifier(
    model=XGBoost_model, nb_classes=2, nb_features = 1*19
)

In [306]:
predictions = classifier.predict(transformed_data)

In [307]:
def convertTarget(x):
    z = np.array(x[:,0] < x[:,1])
    return z.astype(int)

In [308]:
predictions_ = convertTarget(predictions)

In [315]:
attack = ZooAttack(
    classifier= classifier,
    confidence=0.0,
    targeted=False,
    learning_rate=1e-1,
    max_iter=200,
    binary_search_steps=10,
    initial_const=1e-3,
    abort_early=True,
    use_resize=False,
    use_importance=False,
    nb_parallel=5,
    batch_size=1,
    variable_h=0.01,
)


In [316]:
test_data.head()

,id,year,issue_d,final_d,emp_length_int,home_ownership,home_ownership_cat,income_category,annual_inc,income_cat,...,interest_payment_cat,loan_condition,loan_condition_cat,interest_rate,dti,total_pymnt,total_rec_prncp,recoveries,installment,region
0,1077501,2011,01/12/2011,1012015,10.0,RENT,1,Low,24000,1,...,1,Good Loan,0,10.65,27.65,5861.071414,5000.00,0.00,162.87,munster
1,1077430,2011,01/12/2011,1042013,0.5,RENT,1,Low,30000,1,...,2,Bad Loan,1,15.27,1.00,1008.710000,456.46,117.08,59.83,leinster
2,1077175,2011,01/12/2011,1062014,10.0,RENT,1,Low,12252,1,...,2,Good Loan,0,15.96,8.72,3003.653644,2400.00,0.00,84.33,cannught
3,1076863,2011,01/12/2011,1012015,10.0,RENT,1,Low,49200,1,...,2,Good Loan,0,13.49,20.00,12226.302210,10000.00,0.00,339.31,ulster
4,1075358,2011,01/12/2011,1012016,1.0,RENT,1,Low,80000,1,...,1,Good Loan,0,12.69,17.94,3242.170000,2233.10,0.00,67.79,ulster
